### Imports

In [9]:
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from xai_methods import integrated_gradients, lime_explanation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
sys.path.append(os.path.abspath('../../src'))

from helper_functions.path_resolver import DynamicPathResolver
from helper_functions import visualization as visual
from xai_methods import integrated_gradients, lime_explanation

### Paths

In [3]:
resolver = DynamicPathResolver(marker="README.md")
structure = resolver.structure

preprocessed_data_path = structure.data.preprocessed.data_for_gru.phishing_urls_combined_csv
model_path = structure.models.gru.gru_model_best_pth

Project Root: c:\Users\ilian\Documents\Projects\git_projects\university\phishing_detection


In [4]:
data_path = preprocessed_data_path

### Load url data

In [5]:
url_df = pd.read_csv(data_path)
url_df.head()

,url,status
0,1pcables-inox.com/index.php?f=contact.ebay.com...,0
1,https://mb923.net,0
2,https://motoart.kharkov.ua,0
3,'www.leepoo.com.cn/js/?us.battle.net/login/en/...,0
4,https://www.computerhope.com/issues/ch000248.htm,1


### Prepare data

In [6]:
url_df = url_df.head(10)

X = url_df['url'].values
y = url_df['status'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, padding='post')
X_test_pad = pad_sequences(X_test_seq, padding='post')

In [11]:
model = torch.load(model_path)

C:\Users\ilian\AppData\Local\Temp\ipykernel_35344\2457720587.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [16]:
instance = X_test_pad[0]
attributions = integrated_gradients(model, instance.reshape(1, -1))

TypeError: 'collections.OrderedDict' object is not callable

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(range(len(attributions[0])), attributions[0])
plt.title('Integrated Gradients attributions')
plt.xlabel('Feature index')
plt.ylabel('Attribution value')
plt.show()

labels = ["Class 0: Legit", "Class 1: Phishing"]
lime_explanation(model, X_train_pad, X_test_pad, labels)